# Model Experimentation

In [2]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
import hyperopt
from hyperopt import fmin, early_stop, tpe, hp